In [2]:
from pathlib import Path
import json
import re
from tqdm import tqdm

settings = json.loads(Path('../../../settings.json').read_text())preprocessed_data_path = Path(settings['preprocessed_data_path'])data_path = Path('.').resolve()data_name = data_path.namesource_name = data_path.parent.namepreprocessed_dir = preprocessed_data_path/source_name/data_namesource_data_dir = preprocessed_dir/'preprocessed'
splits = ['train', 'valid']
tasks = ['LM(한국어)', '대화 주제 식별', '화자 관계 식별', 'dialog', '화자 인식']
task_data_dir = preprocessed_dir/'preprocessed_task'
task_data_dir.mkdir(exist_ok=True)
for task in tasks:
    task_path = task_data_dir/task
    task_path.mkdir(exist_ok=True)
    
#### prepare for task preprocess
pattern = re.compile(r'\s+')
#### prepare for task preprocess end

#### task preprocess
for split in splits:
    source_data_dir_split = source_data_dir/split
    task_files = [(task_data_dir/task/f'{split}.jsonl').open('w', encoding='utf-8') for task in tasks]
    for source_data in tqdm(list(source_data_dir_split.iterdir()), desc=split):
        source_data = source_data.open()
        for line in source_data.readlines():
            line = json.loads(line)
            
            #### data preprocess
            #### ! utterance 합치기
            #### ! speaker_id가 변경되는 것을 기준으로 utterence를 합침

            new_utt = {"utterence": []}
            new_utt_id_prefix = line['id']+'.1.'
            utt_idx = 1
            for utt in line['utterance']:
                if new_utt['utterence'] == [] or new_utt['utterence'][-1]['speaker_id'] != utt['speaker_id']:
                    new_utt['utterence'].append({   'id': new_utt_id_prefix+str(utt_idx),
                                                    'start_id': utt['id'],
                                                    'end_id': utt['id'],
                                                    'speaker_id': utt['speaker_id'],
                                                    'form': utt['form'],
                                                    'original_form': utt['original_form']})
                    utt_idx += 1
                else:
                    new_utt['utterence'][-1]['original_form'] += ' ' + utt['original_form']
                    new_utt['utterence'][-1]['form'] += ' ' + utt['form']
                    new_utt['utterence'][-1]['end_id'] = utt['id']
            
            based_text = {'original_form':[utt['original_form'].replace('  ', ' ').strip() for utt in new_utt['utterence']],
                            'form':[utt['form'].replace('  ', ' ').strip() for utt in new_utt['utterence']],}
            
            
            #### data preprocess end 

            #### LM(한국어)
            data = {'text': None}
            ## preprocess data from line
            data['text'] = '\n'.join(based_text['form'])

            ## preprocess data from line end
            task_files[0].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### LM(한국어) end


            #### 대화 주제 식별
            data = {'input': None, 'output': None}
            ## preprocess data from line
            data['input'] = based_text
            data['output'] = line['metadata']['topic']
            ## preprocess data from line end
            task_files[1].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### 대화 주제 식별 end


            #### 화자 관계 식별
            data = {'input': None, 'output': None}
            ## preprocess data from line
            data['input'] = {'speaker': line['metadata']['speaker'],
                             'dialog': new_utt['utterence']}
            data['output'] = line['metadata']['setting']['relation']

            ## preprocess data from line end
            task_files[2].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### 화자 관계 식별 end


            #### dialog
            data = {'text': None}
            ## preprocess data from line
            data['text'] = {'metadata' : line['metadata']}
            data['text']['speaker'] = line['metadata']['speaker']
            data['text']['utterence'] = based_text
            data['text']['dialog'] = [utt['speaker_id'] for utt in new_utt['utterence']]        

            ## preprocess data from line end
            task_files[3].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### dialog end


            #### 화자 인식
            data = {'input': None, 'output': None}
            ## preprocess data from line
            data['input'] = {'metadata' : line['metadata']}
            data['input']['speaker'] = line['metadata']['speaker']
            data['input']['utterence'] = [{'id':utt['id'],'original_form':utt['original_form']} for utt in line['utterance']]
            data['output'] = [utt['speaker_id'] for utt in line['utterance']]        

            ## preprocess data from line end
            task_files[4].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### 화자 인식 end


            
    #         break
    #     break
    # break

    for path in task_files:
        path.close()      
        
#### task preprocess end      

train: 100%|██████████| 1/1 [00:04<00:00,  4.48s/it]
valid: 0it [00:00, ?it/s]
